<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_seq_bn_helpfulness_tapt_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This TAPT pretrains a seq_bn adapter on the micro amazon helpfulness dataset.

# Settings
* LR of 0.0001
* No warmup
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* Adapter - seq_bn
  * torch_compile=False (Because no adapter)
* early_stopping_patience=10
* 100 epoch




# Setup

In [21]:
from time import time
start_time = time()

In [22]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

In [23]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset_condensed"

dataset = load_dataset(dataset_name)

print(dataset_name)

BigTMiami/amazon_MICRO_helpfulness_dataset_condensed


In [24]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

adapter_config = SeqBnConfig()
adapter_name = "micro_helpfulness_tapt_pretrain_seq_bn"
adapter_hub_name = f"{adapter_name}_adapter"
print(adapter_hub_name)

micro_helpfulness_tapt_pretrain_seq_bn_adapter


In [26]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

print(f"Adapter Name: {adapter_name}")

Adapter Name: micro_helpfulness_tapt_pretrain_seq_bn


In [27]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
micro_helpfulness_tapt_pretrain_seq_bnbottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [29]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name + "_model",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [31]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [35]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [36]:
train_results = trainer.train()
print(train_results)

Epoch,Training Loss,Validation Loss
1,4.702900,3.798194
2,3.640700,3.171170
3,3.155800,2.818726
4,2.880500,2.609578
5,2.690400,2.452736
6,2.547800,2.356618


Epoch,Training Loss,Validation Loss
1,4.702900,3.798194
2,3.640700,3.171170
3,3.155800,2.818726
4,2.880500,2.609578
5,2.690400,2.452736
6,2.547800,2.356618
7,2.451700,2.263662
8,2.374100,2.226694
9,2.314400,2.173699
10,2.263100,2.135700


TrainOutput(global_step=4042, training_loss=1.9572938091385665, metrics={'train_runtime': 5878.1606, 'train_samples_per_second': 33.582, 'train_steps_per_second': 0.8, 'total_flos': 4.580252807019725e+16, 'train_loss': 1.9572938091385665, 'epoch': 86.0})


In [37]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 1.7229053974151611, 'eval_runtime': 22.3866, 'eval_samples_per_second': 88.848, 'eval_steps_per_second': 4.244, 'epoch': 86.0}


In [38]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/micro_helpfulness_tapt_pretrain_seq_bn_adapter/commit/7d6d9fb8dd01b832102d1f09b8edf8e30891ef21', commit_message='Upload model', commit_description='', oid='7d6d9fb8dd01b832102d1f09b8edf8e30891ef21', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 101 minutes OR 6037 seconds


In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
